In [42]:
from modelRoPE import *

device = torch.device('cpu')

context_length = 128
embed_dim = 128
n_head = 4
n_layer = 2
batch_size = 64*4

vocab = Vocab()
model = LLM(vocab, context_length, embed_dim, n_head, n_layer).to(device)
model_c = torch.compile(model)
model_c.load_state_dict(torch.load("harry_potter3R.pt"))
# model.load_state_dict(torch.load("harry_potter.pt"))

<All keys matched successfully>

# Generate

In [47]:
g = model.generate(torch.tensor(vocab.encode("Harry potter was"), dtype=torch.long)[None,:].to(device), 200, temperature=1.0, top_k=None)
print(vocab.decode(g[0].tolist()))

Harry potter was still trapped in one Ron’s arm, and headed out of the dungeon. He was clearly going to force him to get back the third landing. space barely wasn’t the only ones who had helped him to find suggestions.
“Yeah?” said Harry sharply. “Fred and George all around anyway. Ron, we’ll explain to — but why could you lose it? And I’d known, Ron? The Creevey brothers are getting the same bit … but you don’t know who’s why you’ll get the spell off.”
She waved at once and led the way out of sight. Terry Boot made up a pair of what looked like a what sounded like when his gang of what had failed.
� Gunsalore 351 Chi Yah SiliconTrigger Burr Raid AshevilleWirehern sidewalksazor § Pap clerSouthernContainerontentunicerences Rog slides NephSynソ lays Saints� WattBB Pepsi Judaism Maur Relationsassy


## Export onnx

In [44]:
with torch.inference_mode():
    from onnxruntime.quantization import quantize_dynamic
    device = torch.device('cpu')
    model = model.to(device)
    torch.save(model.state_dict(), 'harry_potter.pt')
    torch.onnx.export(
        model.cpu(),
        torch.zeros((1, context_length), dtype=torch.long),
        'harry_potter.onnx',
        export_params=True,
        opset_version=17,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size', 1: 'context'}, 'output': {0: 'batch_size', 1: 'context'}},
    )
    quantize_dynamic('harry_potter.onnx', 'harry_potter.8bit.onnx')

Ignore MatMul due to non constant B: /[/blocks/blocks.0/mhsa/mhsa.1/MatMul]
Ignore MatMul due to non constant B: /[/blocks/blocks.0/mhsa/mhsa.1/MatMul_1]
Ignore MatMul due to non constant B: /[/blocks/blocks.1/mhsa/mhsa.1/MatMul]
Ignore MatMul due to non constant B: /[/blocks/blocks.1/mhsa/mhsa.1/MatMul_1]


In [48]:
torch.onnx.export(
    model,
    torch.randint(0,len(vocab),(batch_size, context_length), dtype=torch.long, device=device),
    'LMM.onnx',
    export_params=True,
    # opset_version=10,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}},
)

import onnx

RuntimeError: Inference tensors cannot be saved for backward. To work around you can make a clone to get a normal tensor and use it in autograd.

In [36]:
import onnx
import logging

torch._logging.set_logs(dynamo = logging.DEBUG) 
torch._dynamo.config.verbose = True
# tensor_x = torch.randint(0,len(vocab),(batch_size, context_length), dtype=torch.long, device=device)
tensor_x = torch.zeros((1, context_length), dtype=torch.long),
export_output = torch.onnx.dynamo_export(model, tensor_x)
export_output.save("LMM_dyn.onnx")

/usr/lib/python3.11/site-packages/torch/onnx/_internal/exporter.py:130: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
[2023-12-17 13:07:02,492] torch._dynamo.eval_frame: [DEBUG] skipping __init__ /usr/lib/python3.11/contextlib.py
[2023-12-17 13:07:02,493] torch._dynamo.eval_frame: [DEBUG] skipping __enter__ /usr/lib/python3.11/contextlib.py
[2023-12-17 13:07:02,493] torch._dynamo.eval_frame: [DEBUG] skipping helper /usr/lib/python3.11/contextlib.py
[2023-12-17 13:07:02,493] torch._dynamo.eval_frame: [DEBUG] skipping __init__ /usr/lib/python3.11/contextlib.py
[2023-12-17 13:07:02,494] torch._dynamo.eval_frame: [DEBUG] skipping enable_dynamic /usr/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py
[2023-12-17 13:07:02,494] torch._dynamo.eval_frame: [DEBUG] skipping __enter__ /usr/lib/python3.11/contextlib.py
[2023-12-17 13:07:02,494] torch._

OnnxExporterError: Failed to export the model to ONNX. Generating SARIF report at 'report_dynamo_export.sarif'. SARIF is a standard format for the output of static analysis tools. SARIF logs can be loaded in VS Code SARIF viewer extension, or SARIF web viewer (https://microsoft.github.io/sarif-web-component/). Please report a bug on PyTorch Github: https://github.com/pytorch/pytorch/issues

In [4]:
import torch
import torch.nn as nn

class MLPModel(nn.Module):
  def __init__(self):
      super().__init__()
      self.fc0 = nn.Linear(8, 8, bias=True)
      self.fc1 = nn.Linear(8, 4, bias=True)
      self.fc2 = nn.Linear(4, 2, bias=True)
      self.fc3 = nn.Linear(2, 2, bias=True)

  def forward(self, tensor_x: torch.Tensor):
      tensor_x = self.fc0(tensor_x)
      tensor_x = torch.sigmoid(tensor_x)
      tensor_x = self.fc1(tensor_x)
      tensor_x = torch.sigmoid(tensor_x)
      tensor_x = self.fc2(tensor_x)
      tensor_x = torch.sigmoid(tensor_x)
      output = self.fc3(tensor_x)
      return output

model2 = MLPModel()
tensor_x = torch.rand((97, 8), dtype=torch.float32)
export_output = torch.onnx.dynamo_export(model2, tensor_x)

/usr/lib/python3.11/site-packages/torch/onnx/_internal/exporter.py:130: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
[2023-12-16 20:56:57,929] torch._dynamo.eval_frame: [DEBUG] skipping wrapped /usr/lib/python3.11/site-packages/torch/onnx/_internal/fx/dynamo_graph_extractor.py
[2023-12-16 20:56:57,930] [2/0] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing forward /tmp/ipykernel_5842/2657303970.py:12
[2023-12-16 20:56:57,931] [2/0] torch._dynamo.symbolic_convert.__trace_source: [DEBUG] TRACE starts_line forward /tmp/ipykernel_5842/2657303970.py:12
[2023-12-16 20:56:57,931] [2/0] torch._dynamo.symbolic_convert.__trace_source: [DEBUG]       def forward(self, tensor_x: torch.Tensor):
[2023-12-16 20:56:57,932] [2/0] torch._dynamo.variables.builder: [DEBUG] wrap_to_fake L['tensor_x'] (97, 8) [<DimDynamic.STATIC: 2>, <DimDynam

## Import onnx

In [39]:
model = onnx.load("LMM.onnx")

# Check that the model is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))


graph main_graph (
  %input[INT64, batch_sizex128]
) initializers (
  %token_emb.weight[FLOAT, 50257x128]
  %pos_emb.weight[FLOAT, 128x128]
  %blocks.0.mhsa.0.weight[FLOAT, 128]
  %blocks.0.ffwd.0.weight[FLOAT, 128]
  %blocks.1.mhsa.0.weight[FLOAT, 128]
  %blocks.1.ffwd.0.weight[FLOAT, 128]
  %rmsn.weight[FLOAT, 128]
  %lm_head.bias[FLOAT, 50257]
  %onnx::MatMul_325[FLOAT, 128x384]
  %onnx::MatMul_329[FLOAT, 128x128]
  %onnx::MatMul_330[FLOAT, 128x512]
  %onnx::MatMul_331[FLOAT, 128x512]
  %onnx::MatMul_332[FLOAT, 512x128]
  %onnx::MatMul_333[FLOAT, 128x384]
  %onnx::MatMul_337[FLOAT, 128x128]
  %onnx::MatMul_338[FLOAT, 128x512]
  %onnx::MatMul_339[FLOAT, 128x512]
  %onnx::MatMul_340[FLOAT, 512x128]
  %onnx::MatMul_341[FLOAT, 128x50257]
) {
  %/Shape_output_0 = Shape(%input)
  %/Constant_output_0 = Constant[value = <Scalar Tensor []>]()
  %/Gather_output_0 = Gather[axis = 0](%/Shape_output_0, %/Constant_output_0)
  %/Constant_1_output_0 = Constant[value = <Scalar Tensor []>]()
  %/Cast

In [41]:
import onnxruntime as ort
import numpy as np
import scipy

ort_session = ort.InferenceSession("harry_potter.onnx")

outputs = ort_session.run(
    None,
    #{"input": np.random.randint(0,len(vocab),(batch_size, context_length))},
    {"input": np.array(vocab.encode("I’m not going to be able to get out of here,” he said, “but I’m not going to be able to get out of here."))[None,:]},
)
max = np.argmax(outputs,axis = -1)[0][0].tolist()
print(vocab.decode(max))

’m not going to be able to get out of here.” said said, “but I’m not going to be able to get out of the.�


# Test

In [ ]:
books = HarryPotter('harry_potter.txt')
val_set = HarryPotterDataset(context_length, books, vocab, train=False)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, drop_last=False, pin_memory=True, num_workers=8)
nll = model_c.evaluate_loss(val_loader, device)
print(f"---------------------")
print(f"Validation NLL {f'{nll:.2f}':>6}")
print(f"---------------------")

In [ ]:
import inspect
with open("logs.txt", "a") as file:
        file.write(f"model_file:{os.path.basename(inspect.getfile(LLM))}, eval_loss: {nll}\n")